In [4]:
import openai
import langchain

In [ ]:
import os
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
import langchain as lc

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage , HumanMessage , SystemMessage

In [4]:
import pandas as pd
import plotly.express as px
from IPython.display import display,Markdown

In [5]:
electric_cars = pd.read_csv("electric_cars.csv")

print("Description of numeric columns\n")
display(electric_cars.describe())

print("Description of numberic columns\n")
display(electric_cars.describe(include='O'))

print("Electric cars Dataset\n")
electric_cars

Description of numeric columns



,model_year,n_cars
count,26813.000000,26813.000000
mean,2019.375527,5.612166
std,3.286257,26.997325
min,1997.000000,1.000000
25%,2017.000000,1.000000
50%,2020.000000,2.000000
75%,2022.000000,4.000000
max,2024.000000,1514.000000


Description of numberic columns



,city,county,make,model,electric_vehicle_type
count,26813,26813,26813,26813,26813
unique,683,183,37,127,2
top,Bothell,King,TESLA,LEAF,Battery Electric Vehicle (BEV)
freq,479,7066,5071,1889,15885


Electric cars Dataset



,city,county,model_year,make,model,electric_vehicle_type,n_cars
0,Seattle,King,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),1514
1,Seattle,King,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),1153
2,Seattle,King,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),1147
3,Seattle,King,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),1122
4,Bellevue,King,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),931
...,...,...,...,...,...,...,...
26808,Lakewood,Pierce,2022,BMW,IX,Battery Electric Vehicle (BEV),1
26809,Lakewood,Pierce,2022,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),1
26810,Lakewood,Pierce,2022,FORD,TRANSIT,Battery Electric Vehicle (BEV),1
26811,Lakewood,Pierce,2022,HYUNDAI,KONA ELECTRIC,Battery Electric Vehicle (BEV),1


In [ ]:
system_msg_test = """ You are a helpful assistant who understand data science.
You write in a clear language that a ten year old can understand.
You keep your answers brief."""

user_msg_test = "Tell me some uses og gpt for data analysis"

msgs_test = [
    {"role": "system", "content": system_msg_test},
    {"role": "user", "content": user_msg_test}
]
rsps_test = openai.ChatCompletion.create(
    model = "get-3.5-turbo",
    messages = msgs_test
)

In [ ]:
print(rsps_test)

print(rsps_test["choices"][0]['message']['content'])



In [7]:
dataset_description = """
You have a dataset about electric cars registered in Washington state, USA in 2020. It is available as a pandas DataFrame named `electric_cars`.

Each row in the dataset represents the count of the number of cars registered within a city, for a particular model.

The dataset contains the following columns.

- `city` (character): The city in which the registered owner resides.
- `county` (character): The county in which the registered owner resides.
- `model_year` (integer): The [model year](https://en.wikipedia.org/wiki/Model_year#United_States_and_Canada) of the car.
- `make` (character): The manufacturer of the car.
- `model` (character): The model of the car.
- `electric_vehicle_type` (character): Either "Plug-in Hybrid Electric Vehicle (PHEV)" or "Battery Electric Vehicle (BEV)".
- `n_cars` (integer): The count of the number of vehicles registered.
"""
suggest_questions = "Suggest some data analysis questions that could be answered with this dataset."

msgs_suggest_questions = [
    SystemMessage(content='You are data analysis expert.'),
    HumanMessage(content=f"{dataset_description}\n\n{suggest_questions}")
]


In [ ]:
chat = ChatOpenAI()
rsps_suggest_questions = chat(msgs_suggest_questions)

# Print the response
print(rsps_suggest_questions)

print("\n----\n")

# Print just the response's content

print(rsps_suggest_questions.content)

print("\n----\n")

print(type(rsps_suggest_questions))

In [ ]:
msgs_python_top_models = msgs_suggest_questions + [
    rsps_suggest_questions,
    HumanMessage(content="Write some Python code to find the top 5 most popilar make/model combinations of electric car in washington")
]

rsps_python_top_models = chat(msgs_python_top_models)

display(Markdown(rsps_python_top_models.content))

In [ ]:
chat0 =ChatOpenAI(temperature=0)

In [ ]:
msgs_python_plot = msgs_python_top_models +[
    rsps_python_top_models,
    HumanMessage(content='Write some Python code to draw a bar plot of the total count of electric cars by model year, with bars colored by electric vehicle type. Use the Plotly Express package')
]

rsps_python_plot = chat0(msgs_python_plot)

display(Markdown(rsps_python_plot.content))

In [ ]:
# Call GPT again with the same message list and display the response
rsps_python_plot1 = chat0(msgs_python_plot)

display(Markdown(rsps_python_plot1.content))

In [ ]:
# Paste the code generated by GPT and run it

# This time GPT gave a correct answer

import plotly.express as px

# Group the dataset by model year and electric vehicle type, and sum the number of cars for each combination
model_year_counts = electric_cars.groupby(['model_year', 'electric_vehicle_type']).sum('n_cars').reset_index()

# Plot the bar plot using Plotly Express
fig = px.bar(model_year_counts, x='model_year', y='n_cars', color='electric_vehicle_type',
             title='Total Count of Electric Cars by Model Year',
             labels={'model_year': 'Model Year', 'n_cars': 'Count of Electric Cars'},
             color_discrete_map={'Plug-in Hybrid Electric Vehicle (PHEV)': 'blue',
                                 'Battery Electric Vehicle (BEV)': 'green'})

# Show the plot
fig.show()
